In [6]:
import os,pathlib
import pandas as pd 
projectDir=pathlib.Path(os.getcwd()).parent
inputPath=pathlib.Path(projectDir,'data','input','sampledata.csv')
intermediatePath=pathlib.Path(projectDir,'data','intermediate','cleansed_data.csv')

In [7]:

df=pd.read_csv(inputPath,parse_dates=['Date']).convert_dtypes()
#convert tags column to string 
df['Tags']=df['Tags'].astype('string')

In [8]:
#clean Na values 
def cleanNaValues(p_df):
    try:
        for i in p_df.columns:
            p_df[i]=p_df[i].fillna('')
        return p_df
    except Exception as e:
        print('Failed to clean',e)

#add a new column called instruction in dataframe which will served as instruction to LLM
def addLlmInstruction(row):
    if row['type']=='Expense':
        if len(row['Note'])>1:
            return f''' On Date {row['Date']} an amount of rupees {row['Amount']} were spent from {row['Payment mode']} account in category
            "{row['Category']}" with a note mentioned as "{row['Note']}"  '''
        else:
            return f''' On Date {row['Date']} an amount of rupees {row['Amount']} were spent from {row['Payment mode']} account in category
            "{row['Category']}"  '''
    else:
        if len(row['Note'])>1:
            return f''' On Date {row['Date']} a transaction of rupees {row['Amount']} was recieved into {row['Payment mode']} account in category
            "{row['Category']}" with a note mentioned as "{row['Note']}"  '''
        else:
            return f''' On Date {row['Date']} a transaction of rupees {row['Amount']} was recieved into {row['Payment mode']} account in category
            "{row['Category']}"  '''    

#save the formatted csv file in intermediate folder
def savePreprocessedFile(df):    
    df.to_csv(intermediatePath)
    print('Data saved after cleansing')


In [9]:
#main
#call functions one by one 
df=cleanNaValues(df)
df['instructions']=df.apply(addLlmInstruction,axis=1)   
savePreprocessedFile(df) 

Data saved after cleansing


In [10]:
df.head(2)

,Date,Category,Amount,Note,type,Payment mode,To payment mode,Tags,instructions
0,2025-01-31 14:53:00,Items in home,450.0,Washing machine cover,Expense,ICICI BANK,,,On Date 2025-01-31 14:53:00 an amount of rupe...
1,2025-01-31 14:52:00,Dining and eatouts,20.0,Dabeli,Expense,SBI bank,,,On Date 2025-01-31 14:52:00 an amount of rupe...
